In [1]:
import pandas as pd
import numpy as np

#requests for overpass querys
import requests

#to get map from arcgis
from arcgis.gis import GIS

#for environment variables
import os
from dotenv import load_dotenv

#to get villages
from fuzzywuzzy import process

#to extract images
from selenium import webdriver
from ipywidgets.embed import embed_minimal_html

In [2]:
##LOAD ENVIRONMENT from a .env file
load_dotenv()

True

## OVERPASS - OPENSTREETMAPS

get coordinates of some village

#### to learn :
http://osmlab.github.io/learnoverpass//en/

In [3]:
overpass_url = "http://overpass-api.de/api/interpreter"

#### GET ALL VILLAGES IN NIKKI

In [4]:
# got the idea from http://osmlab.github.io/learnoverpass/en/docs/filters/around/
overpass_query= ("""
                    [out:json];
                    area['name'='Nikki']->.a;
                    way[place=village](area.a);
                    (._;>;);
                    out body;
                    """)

response = requests.get(overpass_url,
                    params={
                        "data":overpass_query
                    })

In [5]:
data=response.json()

In [6]:
villages = [elem for elem in data['elements'] 
                 if elem['type'] == 'way']

In [7]:
df_villages = pd.DataFrame(villages)

In [8]:
df_villages['name']=df_villages['tags'].apply(lambda x: 
                                              x['name'] if 'name' in x.keys() 
                                              else None)

In [9]:
vil_names = df_villages['name'].to_list()

### get desired village

In [10]:
#busquemos el pueblo de Sansi Gando - o cualquier otro modificando sansi gando
#string_search="Sansi Gando"
string_search="Alafiarou"
proc = process.extractOne(string_search,vil_names)
print(proc)
vil=proc[0]
vil

('ALAFIAROU', 100)


'ALAFIAROU'

In [11]:
id_vil = df_villages[df_villages['name']==vil]['id'].values[0]
id_vil

737420357

##### by element id 
http://osmlab.github.io/learnoverpass/en/docs/filters/element-id/

In [12]:
## busqueda del id
overpass_query = ("""
[out:json];
area({});
(._;>;);
out body;
""".format(id_vil))

response = requests.get(overpass_url,
                    params={
                        "data":overpass_query
                    })

##### by name
preferred

In [13]:
## cogemos las casas del pueblo para calcular el centro
overpass_query = ("""
[out:json];
area['name'='"""+vil+"""']->.a;
way[building](area.a);
(._;>;);
out body;
""")

response = requests.get(overpass_url,
                    params={
                        "data":overpass_query
                    })

In [14]:
data=response.json()

In [15]:
## get all houses in the town
houses = []
ways=[elem for elem in data['elements']  if elem['type']=='way']
nodes=[elem for elem in data['elements']  if elem['type']=='node']
for way in ways:
    houses_info={}
    houses_info['id']=way['id']
    nodes_lats=[]
    nodes_longs=[]
    houses_info['nodes']=way['nodes']
    for node in nodes:
        if node['id'] in houses_info['nodes']:
            nodes_lats.append(node['lat'])
            nodes_longs.append(node['lon'])
    houses_info['lat']=np.mean(np.array(nodes_lats))
    houses_info['lon']=np.mean(np.array(nodes_longs))
    houses.append(houses_info)

In [16]:
df_houses=pd.DataFrame(houses)
len(df_houses)

114

In [17]:
center=(df_houses.lat.mean(),df_houses.lon.mean())
center

(9.903679592397662, 3.4022777929093575)

### READ FROM ESRI

get token from: https://developers.arcgis.com/dashboard <br>
documentation: https://developers.arcgis.com/python/guide/using-the-map-widget/ <br>

if widget not showing: <br>
jupyter nbextension enable --py --sys-prefix arcgis
jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [18]:
#os.getenv("GOOGLE_API_KEY")

In [19]:
gis = GIS(username=os.environ['ESRI_USERNAME'], password=os.environ['ESRI_PASS'])

In [20]:
ex_map = gis.map(zoomlevel=17)

In [21]:
ex_map.layout={'height':'600px','width':'800px'}

In [22]:
ex_map.center =center
ex_map.basemap = 'satellite'

In [24]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [25]:
ex_map

MapView(layout=Layout(height='600px', width='800px'))

In [27]:
ex_map.export_to_html(vil+'_gmap.html')

True

In [481]:
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [482]:
fig_layout={
        "width":800,
        "height":600
            }

In [504]:
figure_layout = {
    'width': '{}px'.format(fig_layout['width']), #zoom_level_21 this is eq_to 0.002128 longitudes -> each px 2.66e-06 long
    'height': '{}px'.format(fig_layout['height']),#zoom_level_21 this is eq_to 0.00147 longitudes -> each px 2.45e-06 lat
    'border': '1px solid black',
    'padding': '1px'
}
zoom_level=21

In [535]:
#lets_move center for alafiarou
new_center = (center[0]+0.0018,center[1]-0.0014)

In [536]:
fig2 = gmaps.figure(#center=center,
                  center=new_center,
                   zoom_level=zoom_level,
                   map_type='SATELLITE',
                   layout=figure_layout)

In [571]:
### if it doesn't show check jupyter nbextension enable --py gmaps
fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### add houses already mapped

In [538]:
center=new_center

In [539]:
df_houses['dist_center'] = df_houses.apply(lambda x:
                                           np.sqrt(
                                                   (x['lat']-center[0])**2+(x['lon']-center[1])**2
                                                  ),axis=1
                                           )

In [540]:
df_clos_houses=df_houses[df_houses['dist_center']<df_houses['dist_center'].quantile(0.2)]

In [541]:
house_points=[a for a in df_clos_houses.apply(lambda x: (x.lat,x.lon),axis=1)]

In [542]:
marker_layer=gmaps.symbol_layer(
    house_points, fill_color='red', stroke_color='black', scale=3
)

In [543]:
fig2.add_layer(marker_layer)

In [544]:
fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### ADD CORNERS

In [545]:
corners_converstion = {
    21:{
        "latitude":2.46e-06,
        "longitude":2.65e-06
    }
}

In [546]:
#Y=9.947598161535087 X=3.41859563381579
#right
vertical_conv=corners_converstion[zoom_level]["latitude"]*fig_layout['height']/2
horizontal_conv=corners_converstion[zoom_level]["longitude"]*fig_layout['width']/2

upper_right_limit=(center[0]+vertical_conv,center[1]+horizontal_conv)
lower_right_limit=(center[0]-vertical_conv,center[1]+horizontal_conv)

#left
upper_left_limit=(center[0]+vertical_conv,center[1]-horizontal_conv)
lower_left_limit=(center[0]-vertical_conv,center[1]-horizontal_conv)

corners = [upper_right_limit,lower_right_limit,upper_left_limit,lower_left_limit]

In [550]:
marker_layer=gmaps.symbol_layer(
    corners, fill_color='blue', stroke_color='black', scale=3
)

In [551]:
fig2.add_layer(marker_layer)

In [552]:
fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

## EXPORT MAP

## fist export html

In [556]:
fig = gmaps.figure(center=center,
                   zoom_level=zoom_level,
                   map_type='SATELLITE',
                   layout=figure_layout)

In [557]:
embed_minimal_html(vil+'_21.html', views=[fig])

## get html with selenium and screenshot it

In [558]:
## https://chromedriver.chromium.org/downloads - for versions of chrome (uploaded to git version 85)
driver = webdriver.Chrome(executable_path='/Users/daniel/OAN/SID/sid_color_analysis/extract_images/chromedriver')

In [559]:
file = "file:/Users/daniel/OAN/SID/sid_color_analysis/extract_images/"+vil+"_21.html"

In [560]:
driver.get(file)

In [561]:
driver.find_elements_by_class_name("gmaps-toolbar-btn")[0].click()

In [562]:
source="/Users/daniel/Downloads/map.png"

In [563]:
end="/Users/daniel/OAN/SID/sid_color_analysis/extract_images/"+vil+"_21.png"

In [564]:
os.rename(source, end)

### CUT IMAGE

In [565]:
from PIL import Image

In [400]:
def crop(image_path, coords, saved_location):
    """
    @param image_path: The path to the image to edit
    @param coords: A tuple of x/y coordinates (x1, y1, x2, y2)
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image.save(saved_location)

In [566]:
#cut="/Users/daniel/OAN/nikki_map_house_count/all_tog/"+village+'2.png'
cut="/Users/daniel/OAN/SID/sid_color_analysis/extract_images/crop_"+vil+"_21.png"

In [567]:
crop(end,(0, 100, 745, 535),cut)

## FOR MACHINE LEARNING MODEL

lets crop an image of just house

In [462]:
#iloc 0 and 1 are real houses
#one_house=df_houses.sort_values(by='dist_center').reset_index(drop=True).iloc[0]
one_house=df_houses.sort_values(by='dist_center').reset_index(drop=True).iloc[1]

In [463]:
house_lat = one_house['lat']
house_lon = one_house['lon']

In [464]:
#upper_left_limit = (0,0)
pix_house_hoz_center=fig_layout['width']*(upper_left_limit[0]-house_lat)/(fig_layout['width']*corners_converstion[zoom_level]['latitude'])
pix_house_ver_center=fig_layout['height']*(house_lon-upper_left_limit[1])/(fig_layout['height']*corners_converstion[zoom_level]['longitude'])
print(pix_house_hoz_center,pix_house_ver_center)

441.21403865300226 508.458937437641


In [465]:
cut="/Users/daniel/OAN/SID/sid_color_analysis/extract_images/second_house.png"

In [466]:
box=40

In [467]:
left_cut=pix_house_hoz_center-box
rigth_cut=pix_house_hoz_center+box
upp_cut=pix_house_ver_center-box
lower_cut=pix_house_ver_center+box

In [468]:
crop(end,(upp_cut, left_cut, lower_cut, rigth_cut),cut)

## lets get all

In [568]:
df_sort_houses = df_houses.sort_values(by='dist_center').reset_index(drop=True)
box=40

In [570]:
for i in range(len(df_clos_houses)):
    #get house
    one_house=df_sort_houses.iloc[i]
    house_lat = one_house['lat']
    house_lon = one_house['lon']
    
    #transform to pixels
    pix_house_hoz_center=fig_layout['width']*(upper_left_limit[0]-house_lat)/(fig_layout['width']*corners_converstion[zoom_level]['latitude'])
    pix_house_ver_center=fig_layout['height']*(house_lon-upper_left_limit[1])/(fig_layout['height']*corners_converstion[zoom_level]['longitude'])

    cut="/Users/daniel/OAN/SID/sid_color_analysis/extract_images/houses/"+vil+"/{}_house.png".format(i)
    
    left_cut=pix_house_hoz_center-box
    rigth_cut=pix_house_hoz_center+box
    upp_cut=pix_house_ver_center-box
    lower_cut=pix_house_ver_center+box
    
    crop(end,(upp_cut, left_cut, lower_cut, rigth_cut),cut)